In [1]:
import requests
import json
import pandas as pd

In [2]:
response = requests.get('https://acnhapi.com/v1/villagers')

In [3]:
data = response.json()

In [4]:
print(data.keys())

dict_keys(['ant00', 'ant01', 'ant02', 'ant03', 'ant06', 'ant08', 'ant09', 'bea00', 'bea01', 'bea02', 'bea03', 'bea05', 'bea06', 'bea07', 'bea08', 'bea09', 'bea10', 'bea11', 'bea12', 'bea13', 'bea14', 'bea15', 'brd00', 'brd01', 'brd02', 'brd03', 'brd04', 'brd05', 'brd06', 'brd08', 'brd11', 'brd15', 'brd16', 'brd17', 'brd18', 'bul00', 'bul01', 'bul03', 'bul05', 'bul07', 'bul08', 'cat00', 'cat01', 'cat02', 'cat03', 'cat04', 'cat05', 'cat06', 'cat07', 'cat08', 'cat09', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat23', 'cbr00', 'cbr01', 'cbr02', 'cbr03', 'cbr04', 'cbr05', 'cbr06', 'cbr07', 'cbr09', 'cbr10', 'cbr13', 'cbr14', 'cbr15', 'cbr16', 'cbr17', 'cbr19', 'chn00', 'chn01', 'chn02', 'chn05', 'chn09', 'chn10', 'chn11', 'chn12', 'chn13', 'cow00', 'cow01', 'cow06', 'cow07', 'crd00', 'crd01', 'crd02', 'crd04', 'crd06', 'crd07', 'crd08', 'der00', 'der01', 'der02', 'der03', 'der04', 'der05', 'der06', 'der07', 'der08', 'der09',

In [5]:
data_list = []

In [6]:
for key in data.keys():
    # Each key corresponds to a villager, and the value is another dictionary with the data for that villager
    villager_data = data[key]
    
    # Convert the data for the current villager into a DataFrame
    villager_df = pd.json_normalize(villager_data)
    
    # Append the DataFrame to data_list
    data_list.append(villager_df)
    
villagers = pd.concat(data_list, ignore_index=True)

In [7]:
print(villagers.columns)

Index(['id', 'file-name', 'personality', 'birthday-string', 'birthday',
       'species', 'gender', 'subtype', 'hobby', 'catch-phrase', 'icon_uri',
       'image_uri', 'bubble-color', 'text-color', 'saying', 'name.name-USen',
       'name.name-EUen', 'name.name-EUde', 'name.name-EUes', 'name.name-USes',
       'name.name-EUfr', 'name.name-USfr', 'name.name-EUit', 'name.name-EUnl',
       'name.name-CNzh', 'name.name-TWzh', 'name.name-JPja', 'name.name-KRko',
       'name.name-EUru', 'catch-translations.catch-USen',
       'catch-translations.catch-EUen', 'catch-translations.catch-EUde',
       'catch-translations.catch-EUes', 'catch-translations.catch-USes',
       'catch-translations.catch-EUfr', 'catch-translations.catch-USfr',
       'catch-translations.catch-EUit', 'catch-translations.catch-EUnl',
       'catch-translations.catch-CNzh', 'catch-translations.catch-TWzh',
       'catch-translations.catch-JPja', 'catch-translations.catch-KRko',
       'catch-translations.catch-EUru'],


In [8]:
columns_to_drop = ['file-name','icon_uri', 'image_uri', 'name.name-EUen', 'name.name-EUde', 'name.name-EUes', 'name.name-USes',
       'name.name-EUfr', 'name.name-USfr', 'name.name-EUit', 'name.name-EUnl',
       'name.name-CNzh', 'name.name-TWzh', 'name.name-JPja', 'name.name-KRko',
       'name.name-EUru', 'catch-translations.catch-USen',
       'catch-translations.catch-EUen', 'catch-translations.catch-EUde',
       'catch-translations.catch-EUes', 'catch-translations.catch-USes',
       'catch-translations.catch-EUfr', 'catch-translations.catch-USfr',
       'catch-translations.catch-EUit', 'catch-translations.catch-EUnl',
       'catch-translations.catch-CNzh', 'catch-translations.catch-TWzh',
       'catch-translations.catch-JPja', 'catch-translations.catch-KRko',
       'catch-translations.catch-EUru']
villagers = villagers.drop(columns=columns_to_drop)

In [9]:
villagers.head()

,id,personality,birthday-string,birthday,species,gender,subtype,hobby,catch-phrase,bubble-color,text-color,saying,name.name-USen
0,1,Cranky,March 9th,9/3,Anteater,Male,B,Education,ah-CHOO,#194c89,#fffad4,Don't punch your nose to spite your face.,Cyrano
1,2,Jock,October 20th,20/10,Anteater,Male,B,Fitness,honk,#fff98f,#879b96,Always go for the burn!,Antonio
2,3,Peppy,November 9th,9/11,Anteater,Female,B,Fashion,snooooof,#00d1bd,#fffce9,A thing of beauty is a joy forever.,Pango
3,4,Peppy,February 16th,16/2,Anteater,Female,B,Fashion,snorty,#ff791f,#fff2bb,Dance like no one's watching!,Anabelle
4,5,Snooty,October 24th,24/10,Anteater,Female,A,Education,sniffff,#f2bdc7,#634b4b,"Request forgiveness, not permission.",Snooty


In [10]:
cols = villagers.columns.tolist()
cols.insert(1, cols.pop(cols.index('name.name-USen')))  # remove 'B' from its current location and insert at position 0
villagers = villagers.reindex(columns=cols)  # rearrange the columns

In [11]:
villagers.head()

,id,name.name-USen,personality,birthday-string,birthday,species,gender,subtype,hobby,catch-phrase,bubble-color,text-color,saying
0,1,Cyrano,Cranky,March 9th,9/3,Anteater,Male,B,Education,ah-CHOO,#194c89,#fffad4,Don't punch your nose to spite your face.
1,2,Antonio,Jock,October 20th,20/10,Anteater,Male,B,Fitness,honk,#fff98f,#879b96,Always go for the burn!
2,3,Pango,Peppy,November 9th,9/11,Anteater,Female,B,Fashion,snooooof,#00d1bd,#fffce9,A thing of beauty is a joy forever.
3,4,Anabelle,Peppy,February 16th,16/2,Anteater,Female,B,Fashion,snorty,#ff791f,#fff2bb,Dance like no one's watching!
4,5,Snooty,Snooty,October 24th,24/10,Anteater,Female,A,Education,sniffff,#f2bdc7,#634b4b,"Request forgiveness, not permission."


In [12]:
paradise = pd.read_csv('data/Paradise Planning.csv')
paradise.head()

,Name,Filename,Request,Thought bubble,Song,Furniture List,Furniture Name List,Unique Entry ID
0,Ace,brd09,The Fishin' Hole,I want a place where I can fish to my heart's ...,To the Edge,3064;13637;13347,cooler box;outdoor folding chair;yacht,yrzvoz4kNmqN8N2pX
1,Admiral,brd06,Cool Country Cabin,I want a cool cabin where I can relax on hot d...,K.K. Rally,2736;2326;5979,retro fan;water pump;bamboo noodle slide,2dKeMSMjBLekssvwo
2,Agent S,squ05,Training Grounds for a Hero,"Heroes train! It's what they do, even on vacat...",Go K.K. Rider,7653;4015;4078,speed bag;shower booth;barbell,EN7dGxBHogyTYD4pS
3,Agnes,pig17,The Artsiest Art Museum,I want a place that's like a museum with artsy...,K.K. Fugue,1712;13255;13302,poster stand;elaborate display case;decorative...,pmmgTrYA9hvoj2v9b
4,Al,gor08,Gymnastics School for Kids,My dream is to someday teach gymnastics to kids.,K.K. Étude,12701;13188;7845,school locker;ball catcher;pull-up-bar stand,XKm6cmQjf4eFAHMRT


In [13]:
popularity = pd.read_csv('data/acnh_villager_data.csv')
popularity.head()

,tier,rank,name
0,1,1,Raymond
1,1,2,Marshal
2,1,3,Shino
3,1,4,Sherb
4,1,5,Sasha


In [14]:
poster = pd.read_csv('data/posters.csv')
poster.head()

,Name,Buy,Sell,Color 1,Color 2,Size,Source,Source Notes,Version,Catalog,Filename,Internal ID,Unique Entry ID
0,Admiral's poster,1000,250,Green,Blue,1x1,Nook's Shopping - Posters,"In the Special Items section in Nook Shopping,...",1.0.0,For sale,PosterNpcNmlBrd06,10814,3LW8HyqLZTJRaGQT9
1,Agent S's poster,1000,250,Blue,Gray,1x1,Nook's Shopping - Posters,"In the Special Items section in Nook Shopping,...",1.0.0,For sale,PosterNpcNmlSqu05,10425,bZu92nbNfGmQvp54D
2,Agnes's poster,1000,250,Pink,Black,1x1,Nook's Shopping - Posters,"In the Special Items section in Nook Shopping,...",1.0.0,For sale,PosterNpcNmlPig17,10401,2wZBfe8Ed7QMvQ8tX
3,Al's poster,1000,250,Red,Light blue,1x1,Nook's Shopping - Posters,"In the Special Items section in Nook Shopping,...",1.0.0,For sale,PosterNpcNmlGor08,10590,sat2PG8L97DymZbQw
4,Alfonso's poster,1000,250,Orange,Red,1x1,Nook's Shopping - Posters,"In the Special Items section in Nook Shopping,...",1.0.0,For sale,PosterNpcNmlCrd00,10382,NjW35Zc7MnTipYZbg
